<a href="https://colab.research.google.com/github/nikhildr22/Speech-Emotion-Recognition/blob/master/master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
!ls

sample_data


In [3]:
!wget https://zenodo.org/record/1188976/files/Audio_Speech_Actors_01-24.zip

--2020-04-05 00:51:25--  https://zenodo.org/record/1188976/files/Audio_Speech_Actors_01-24.zip
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 208468073 (199M) [application/octet-stream]
Saving to: ‘Audio_Speech_Actors_01-24.zip’

Audio_Speech_Actors 100%[===================>] 198.81M  9.47MB/s    in 26s     

2020-04-05 00:51:54 (7.62 MB/s) - ‘Audio_Speech_Actors_01-24.zip’ saved [208468073/208468073]



In [4]:
!mkdir dataset
!ls

Audio_Speech_Actors_01-24.zip  dataset	sample_data


In [0]:
!unzip -q Audio_Speech_Actors_01-24.zip -d dataset

In [6]:
!ls

Audio_Speech_Actors_01-24.zip  dataset	sample_data


In [7]:
!pip install soundfile

In [0]:
import librosa
from tqdm import tqdm
import soundfile
from scipy.io import wavfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [0]:
''' directory to keep clean files '''
!mkdir clean 

In [10]:
''' downsampling audio files to 16KHz and storing them in /clean directory ''' 
for f in tqdm(glob.glob("dataset/*/*")):
    signal, rate = librosa.load(path=f, sr=16000)
    wavfile.write(filename='clean/'+f[-24:],rate=rate,data=signal)

100%|██████████| 1440/1440 [05:38<00:00,  4.26it/s]


In [0]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [0]:
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#DataFlair - Emotions to observe (remove the prefix "/" to include that emotion)
observed_emotions=['sad','/angry','happy','/neutral','/calm','/fearful','/disgust','/surprised']

In [0]:
#DataFlair - Load the data and extract features for each sound file

def load_data(test_size=0.2):
    x,y=[],[] 
    for file in tqdm(glob.glob("clean/*")):
        file_name=os.path.basename(file[-24:])
#         print(file_name)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        try:
            feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
            x.append(feature)
            y.append(int(file_name.split("-")[2]))
        except:continue
    return train_test_split(np.array(x), np.array(y), test_size=test_size, random_state=2)

In [31]:
#DataFlair - Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.3)

100%|██████████| 1440/1440 [00:16<00:00, 89.89it/s]


In [32]:
#DataFlair - Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(268, 116)


In [33]:
#DataFlair - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [0]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500,verbose=2)

In [35]:
#DataFlair - Train the model
model.fit(x_train,y_train)

Iteration 1, loss = 6.47672061
Iteration 2, loss = 4.73345911
Iteration 3, loss = 3.60495583
Iteration 4, loss = 1.55426477
Iteration 5, loss = 2.29027900
Iteration 6, loss = 1.17447254
Iteration 7, loss = 2.03217486
Iteration 8, loss = 1.48114861
Iteration 9, loss = 1.05965329
Iteration 10, loss = 1.74578806
Iteration 11, loss = 1.10336884
Iteration 12, loss = 0.89829674
Iteration 13, loss = 1.05553256
Iteration 14, loss = 0.58705459
Iteration 15, loss = 0.74764249
Iteration 16, loss = 0.50278385
Iteration 17, loss = 0.72627182
Iteration 18, loss = 0.61337038
Iteration 19, loss = 0.57560469
Iteration 20, loss = 0.53015663
Iteration 21, loss = 0.61239681
Iteration 22, loss = 0.52317836
Iteration 23, loss = 0.58032864
Iteration 24, loss = 0.48609679
Iteration 25, loss = 0.57704737
Iteration 26, loss = 0.44935929
Iteration 27, loss = 0.53837876
Iteration 28, loss = 0.47901713
Iteration 29, loss = 0.46842358
Iteration 30, loss = 0.42238822
Iteration 31, loss = 0.53052427
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=2, warm_start=False)

In [0]:
#DataFlair - Predict for the test set
y_pred=model.predict(x_test)

In [37]:
#DataFlair - Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
#DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 68.97%


In [38]:
x_train.shape

(268, 180)

In [39]:
y_train.shape

(268,)

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Conv1D,Flatten# Neural network
input_shape = (180,1)
num_files = x_train.shape[0]
x_train = x_train.reshape(num_files,180,1)
y_train = y_train.reshape(num_files)
model0 = Sequential()
model0.add(Conv1D(32, kernel_size=(3), input_shape=input_shape))
model0.add(Conv1D(64, kernel_size=(3)))
model0.add(Conv1D(128, kernel_size=(3)))
model0.add(Conv1D(64, kernel_size=(3)))
model0.add(Conv1D(32, kernel_size=(3)))
model0.add(Flatten())
model0.add(Dense(128, activation='relu'))
model0.add(Dense(12, activation='relu'))
model0.add(Dense(9, activation='softmax'))

In [41]:
model0.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model0.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 178, 32)           128       
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 176, 64)           6208      
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 174, 128)          24704     
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 172, 64)           24640     
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 170, 32)           6176      
_________________________________________________________________
flatten_2 (Flatten)          (None, 5440)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)              

In [42]:
model0.fit(x_train, y_train, epochs=85,verbose=1, batch_size=10)

Epoch 1/85
268/268 [==============================] - 1s 2ms/step - loss: 0.8305 - acc: 0.5896
Epoch 2/85
268/268 [==============================] - 0s 564us/step - loss: 0.5821 - acc: 0.7239
Epoch 3/85
268/268 [==============================] - 0s 547us/step - loss: 0.4694 - acc: 0.7649
Epoch 4/85
268/268 [==============================] - 0s 540us/step - loss: 0.4196 - acc: 0.8134
Epoch 5/85
268/268 [==============================] - 0s 533us/step - loss: 0.3269 - acc: 0.8507
Epoch 6/85
268/268 [==============================] - 0s 526us/step - loss: 0.2678 - acc: 0.8806
Epoch 7/85
268/268 [==============================] - 0s 524us/step - loss: 0.2751 - acc: 0.8881
Epoch 8/85
268/268 [==============================] - 0s 534us/step - loss: 0.2195 - acc: 0.8993
Epoch 9/85
268/268 [==============================] - 0s 508us/step - loss: 0.1778 - acc: 0.9291
Epoch 10/85
268/268 [==============================] - 0s 510us/step - loss: 0.2344 - acc: 0.9366
Epoch 11/85
268/268 [==========

In [0]:
num_testfiles = y_test.shape[0]

In [0]:
x_test = x_test.reshape(num_testfiles,180,1)
y_test = y_test.reshape(num_testfiles)

In [45]:
model0.evaluate(x_test, y_test)

116/116 [==============================] - 0s 920us/step


[1.4978998529499974, 0.8103448275862069]

In [46]:
model0.predict(x_test[3].reshape(1,180,1)).argmax()

3